# YouTube Trending Videos Analysis

### Importing required libraries:

In [1]:
import pandas as pd
import numpy as np
import warnings
import regex as re
warnings.filterwarnings('ignore')#to filter all the warnings
import seaborn as sns
pd.set_option('float_format', '{:.4f}'.format)# to keep the float values short
# Import for wordcloud
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

#import fot plotly
import plotly.express as px

## Dataset Description

### YouTube Trending Statistics - Consists of updated daily records of top trending YouTube videos, based on viewership metrics such as likes, dislikes and views and provides interesting parameters such as tag or country for further analysis.

 


### File type: csv
<ol>
    <li>video_id: Uniquely identifies each video</li>
    <li>published_at: Date and Time of video published</li>
    <li>categoryId: Id of category the video belongs to</li>
    <li>trending_date: Date and time when the video got to Trending</li>
    <li>view_count: Number of views (cumulative)</li>
    <li>likes: Number of Likes(cumulative)</li>
    <li>dislikes: Number of dislikes(cumulative)</li>
    <li>comment_count: Number of comments(cumulative)</li>
    <li>country: Country in which the video was trending</li>
    <li>description: Description of video by the creator</li>
    <li>tags: Tags of the video by the creator</li>
    <li>title: Title of the video</li>
    <li>channelTitle: Channel Title of the video</li>
    <li>thumbnail_link:link for thumbnails</li>
    <li>comments_disabled: boolean value that defines if viewer can comment</li>
    <li>ratings_disabled: boolean value that defines if viewer can rate through likes and dislikes</li>
    <li>channelId: uniquely defines the channel the video is coming from</li>
</ol>

### File type: json
<ol>
<li>id: Id of category the video belongs to</li>
<li>name: Respective category names of category ids</li>  
</ol>


### Importing individual .csv file of USA, Great Britain, Canada from Home directory:

In [2]:

US_Videos_df = pd.read_csv(r'US_youtube_trending_data.csv')
US_Videos_df.head(1)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...


In [3]:
CA_Videos_df = pd.read_csv(r'CA_youtube_trending_data.csv')
CA_Videos_df.head(1)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,KX06ksuS6Xo,Diljit Dosanjh: CLASH (Official) Music Video |...,2020-08-11T07:30:02Z,UCZRdNleCgW-BGUJf-bbjzQg,Diljit Dosanjh,10,2020-08-12T00:00:00Z,clash diljit dosanjh|diljit dosanjh|diljit dos...,9140911,296541,6180,30059,https://i.ytimg.com/vi/KX06ksuS6Xo/default.jpg,False,False,CLASH official music video performed by DILJIT...


In [4]:
GB_Videos_df = pd.read_csv(r'GB_youtube_trending_data.csv')
GB_Videos_df.head(1)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353790,2628,40228,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...


### Finding the shape of these 3 dataframes

In [5]:
print('Shape of GB File: '+ str(GB_Videos_df.shape))
print('Shape of CA File: '+ str(CA_Videos_df.shape))
print('Shape of US File: '+ str(US_Videos_df.shape))

Shape of GB File: (133994, 16)
Shape of CA File: (133943, 16)
Shape of US File: (133990, 16)


### Opening JSON file and loading the required data to match categoryId to its respective category name:

The dataset we downloaded from Kaggle has two types of files for each country, one is 'video.csv' which contains all the features described before, and the other is 'category.json' which contains mapping for category id to category names.

To merge this information together, we did the following steps:

**1. Load JSON File for each country**

In [6]:
import json #import data using python json module
with open(r'US_category_id.json','r') as f:
    category_data_us= json.loads(f.read())

In [7]:
with open(r'CA_category_id.json','r') as f:
    category_data_ca= json.loads(f.read())

In [8]:
with open(r'GB_category_id.json','r') as f:
    category_data_gb= json.loads(f.read())

**2. Since JSON file was in nested format we used json normalize function from pandas to flatten it and read into data frame**



In [9]:
US_cat = pd.json_normalize(category_data_us,record_path='items')
CA_cat = pd.json_normalize(category_data_ca,record_path='items')
GB_cat = pd.json_normalize(category_data_gb,record_path='items')
US_cat

,kind,etag,id,snippet.title,snippet.assignable,snippet.channelId
0,youtube#videoCategory,IfWa37JGcqZs-jZeAyFGkbeh6bc,1,Film & Animation,True,UCBR8-60-B28hp2BmDPdntcQ
1,youtube#videoCategory,5XGylIs7zkjHh5940dsT5862m1Y,2,Autos & Vehicles,True,UCBR8-60-B28hp2BmDPdntcQ
2,youtube#videoCategory,HCjFMARbBeWjpm6PDfReCOMOZGA,10,Music,True,UCBR8-60-B28hp2BmDPdntcQ
3,youtube#videoCategory,ra8H7xyAfmE2FewsDabE3TUSq10,15,Pets & Animals,True,UCBR8-60-B28hp2BmDPdntcQ
4,youtube#videoCategory,7mqChSJogdF3hSIL-88BfDE-W8M,17,Sports,True,UCBR8-60-B28hp2BmDPdntcQ
5,youtube#videoCategory,0Z6uGkj97NgjD-X3pkA-nL18Hqk,18,Short Movies,False,UCBR8-60-B28hp2BmDPdntcQ
6,youtube#videoCategory,K_-7stg0kIU7eUBOPUa6j5fqIMQ,19,Travel & Events,True,UCBR8-60-B28hp2BmDPdntcQ
7,youtube#videoCategory,I3IL9xGIM3MsULlqR4tvCsEKv98,20,Gaming,True,UCBR8-60-B28hp2BmDPdntcQ
8,youtube#videoCategory,D1W6tq5mMMCV0wtNxf9A6g9wWjU,21,Videoblogging,False,UCBR8-60-B28hp2BmDPdntcQ
9,youtube#videoCategory,QMEBz6mxVdklVaq8JwesPEw_4nI,22,People & Blogs,True,UCBR8-60-B28hp2BmDPdntcQ


In [10]:
# Converting the 'id' extracted from the json file to type 'int'
US_cat['id']= US_cat['id'].astype(int)
CA_cat['id']= CA_cat['id'].astype(int)
GB_cat['id']= GB_cat['id'].astype(int)

**3. Merging videos dataframe and category dataframe for all countries using left join**

In [11]:
US_Videos_df= US_Videos_df.merge(US_cat,how ='left',left_on= 'categoryId',\
                                 right_on='id').rename(columns= {'snippet.title':'category_name'})
CA_Videos_df= CA_Videos_df.merge(CA_cat,how ='left',left_on= 'categoryId',\
                                 right_on='id').rename(columns= {'snippet.title':'category_name'})
GB_Videos_df= GB_Videos_df.merge(GB_cat,how ='left',left_on= 'categoryId',\
                                 right_on='id').rename(columns= {'snippet.title':'category_name'})

### Noting the shape again after the merge

In [12]:
print('Shape of GB File: '+ str(GB_Videos_df.shape))
print('Shape of CA File: '+ str(CA_Videos_df.shape))
print('Shape of US File: '+ str(US_Videos_df.shape))

Shape of GB File: (133994, 22)
Shape of CA File: (133943, 22)
Shape of US File: (133990, 22)


### Adding a column 'country' to identify country specific information after appending the 3 countries - USA, Great Britain, Canada

In [13]:
US_Videos_df['country']= 'USA'
CA_Videos_df['country']= 'Canada'
GB_Videos_df['country']= 'Great Britain'

In [14]:
US_Videos_df.head(1)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,...,comments_disabled,ratings_disabled,description,kind,etag,id,category_name,snippet.assignable,snippet.channelId,country
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,...,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,youtube#videoCategory,QMEBz6mxVdklVaq8JwesPEw_4nI,22,People & Blogs,True,UCBR8-60-B28hp2BmDPdntcQ,USA


### Appending Data from 3 countries:

#### Making a list of all countries and using pd.concat function to append data for all countries in one dataframe

In [15]:
df_list= [US_Videos_df,CA_Videos_df,GB_Videos_df]

In [16]:
df= pd.concat(df_list).reset_index(drop=True)

In [17]:
df.groupby('country')['video_id'].count()

country
Canada           133943
Great Britain    133994
USA              133990
Name: video_id, dtype: int64

In [18]:
assert (df.shape[0] == df.groupby('country')['video_id'].count().sum())

# Data Cleaning

#### Dropping columns that are irrelevant for analysis:

In [19]:
df.drop(columns=['thumbnail_link','kind','etag','id','snippet.assignable','snippet.channelId','channelId'], axis='columns').shape

(401927, 16)

#### The above code is a proof that the columns mentioned are safely dropped without disturbing the rows.So,using inplace= True parameter with drop

In [20]:
df.drop(columns=['kind','etag','id','snippet.assignable',\
                 'snippet.channelId','channelId'], axis='columns',inplace=True)

In [21]:
df.head(1)

,video_id,title,publishedAt,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,category_name,country
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,People & Blogs,USA


### Data coherency

#### Finding if there are any True values in comments_disabled and ratings_disabled columns, which represents that comment_count is 0 if  comments_disabled is True, and likes ,dislikes should be 0 if ratings_disabled is True

In [22]:
df[df.comments_disabled==True]['comment_count'].head()

53     0
131    0
134    0
242    0
276    0
Name: comment_count, dtype: int64

In [23]:
df[df.ratings_disabled==True][['likes','dislikes']].sample(5)

,likes,dislikes
366709,0,0
220484,0,0
177845,0,0
65985,0,0
98598,0,0


In [24]:
df[df.comments_disabled==True].comment_count.sum()

0

In [25]:
df[df.ratings_disabled==True][['likes','dislikes']].sum()

likes       0
dislikes    0
dtype: int64

##### The records with True values in Comments_disabled and ratings_disabled have 0 corresponding values(likes, dislikes,comment_count).Hence the data is coherent and it is safe to drop the above 2 columns

In [26]:
df.drop(columns=['comments_disabled','ratings_disabled'],axis=1,inplace=True)

In [27]:
df.shape

(401927, 15)

### Checking for Null values

In [28]:
df.isna().sum()

video_id              0
title                 0
publishedAt           0
channelTitle          0
categoryId            0
trending_date         0
tags                  0
view_count            0
likes                 0
dislikes              0
comment_count         0
thumbnail_link        0
description       11076
category_name       198
country               0
dtype: int64

#### There are null values in category_name and description
#### 1. Dealing with Category_name null values 
#### 2. Dealing with description null values

#### 1. Dealing with null values in category_name

In [29]:
df[df.category_name.isna()].head()

,video_id,title,publishedAt,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,description,category_name,country
138473,zvxrdwbmnHU,Nisswa Mayor Arrested for Involvement in Traff...,2020-09-01T04:50:02Z,Lakeland PBS,29,2020-09-03T00:00:00Z,Lakeland PBS|LPTV|PBS|Bemidji|Minnesota|MN,345993,5829,172,3209,https://i.ytimg.com/vi/zvxrdwbmnHU/default.jpg,"Originally aired August 31, 2020For more infor...",NaN,Canada
138706,zvxrdwbmnHU,Nisswa Mayor Arrested for Involvement in Traff...,2020-09-01T04:50:02Z,Lakeland PBS,29,2020-09-04T00:00:00Z,Lakeland PBS|LPTV|PBS|Bemidji|Minnesota|MN,408941,6578,187,3533,https://i.ytimg.com/vi/zvxrdwbmnHU/default.jpg,"Originally aired August 31, 2020For more infor...",NaN,Canada
138924,zvxrdwbmnHU,Nisswa Mayor Arrested for Involvement in Traff...,2020-09-01T04:50:02Z,Lakeland PBS,29,2020-09-05T00:00:00Z,Lakeland PBS|LPTV|PBS|Bemidji|Minnesota|MN,447580,7233,198,3828,https://i.ytimg.com/vi/zvxrdwbmnHU/default.jpg,"Originally aired August 31, 2020For more infor...",NaN,Canada
139154,zvxrdwbmnHU,Nisswa Mayor Arrested for Involvement in Traff...,2020-09-01T04:50:02Z,Lakeland PBS,29,2020-09-06T00:00:00Z,Lakeland PBS|LPTV|PBS|Bemidji|Minnesota|MN,469103,7607,208,4000,https://i.ytimg.com/vi/zvxrdwbmnHU/default.jpg,"Originally aired August 31, 2020For more infor...",NaN,Canada
139270,P5urIeEcuvA,Miley Cyrus quits veganism,2020-09-05T18:17:10Z,Earthling Ed,29,2020-09-07T00:00:00Z,earthlinged|earthling ed|vegan|veganism|why|be...,132065,14258,380,2822,https://i.ytimg.com/vi/P5urIeEcuvA/default.jpg,✺ find out more about what i do & sign up for ...,NaN,Canada


In [30]:
df[df.categoryId==29].category_name

4474      Nonprofits & Activism
4690      Nonprofits & Activism
4897      Nonprofits & Activism
5132      Nonprofits & Activism
5383      Nonprofits & Activism
                  ...          
389407                      NaN
389634                      NaN
389849                      NaN
390070                      NaN
390312                      NaN
Name: category_name, Length: 312, dtype: object

In [31]:
df[df.category_name.isna()]['title'].unique()

array(['Nisswa Mayor Arrested for Involvement in Traffic Stop',
       'Miley Cyrus quits veganism',
       'Fast Times At Ridgemont High | Virtual Table Read for CORE',
       '[Full livestream] Watch the Countdown Global Launch, a call to action on climate change',
       'RETURNING To YouTube? Our CHARITY FUNDRAISER! (Ft. Orla Gartland)',
       'It Counts', 'Happy Birthday, #TeamTrees!',
       'The cast of Glee pays tribute to Naya Rivera & Santana Lopez at the 32nd Annual GLAAD Media Awards',
       'Color the Spectrum LIVE- Mark Rober and Jimmy Kimmel',
       'Global Citizen VAX Live - Extended Concert Only on YouTube',
       "I Was Hacked. But Now I'm BACK!",
       'President Moon Jae-in & BTS at the Sustainable Development Goals Moment | United Nations (English)',
       'BTS Shine Spotlight on the United Nations as Envoys of the President of the Republic of Korea',
       'BTS (방탄소년단) at Global Citizen Live Concert - Permission to Dance | #GlobalCitizenLive',
       'Coldp

#### For categoryID=29 , USA has a category name NonProfits&Activism whereas CA,GB did not define a categoryname. By taking a look at the Video title of those records,we can come to a conclusion that they represent  NonProfits&Activists category. So, replacing Nan's with NonProfits&Activism.

In [32]:
df.category_name.fillna('Nonprofits & Activism').isna().any()

False

In [33]:
df.category_name.fillna('Nonprofits & Activism',inplace=True)

#### 2. Dealing with Null values in description
##### filling missing values in description with ''

In [34]:
df.description.fillna('')

0         SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
1         While running her own modding shop, Ramya Pare...
2         I left youtube for a month and this is what ha...
3         Subscribe to XXL → http://bit.ly/subscribe-xxl...
4         Transforming The LaBrant Family's empty white ...
                                ...                        
401922    It is time for the new Era. Watch the trailer ...
401923    ►How I Stole Max Hearts On This Minecraft SMP....
401924    The much anticipated and belated patch 2.7 has...
401925    Welcome your faces back to a new dawn and a ne...
401926    Thanks to ExpressVPN for sponsoring this video...
Name: description, Length: 401927, dtype: object

In [35]:
df.description.fillna('').isna().any()

False

In [36]:
df.description.fillna('',inplace=True)

#### Understanding data in descriptive columns

In [37]:
df.tags.sample(4)

149881                                               [None]
230417    Technoblade|Technothepig|Technoblade Minecraft...
245421    Michelob ULTRA|ultra light beer|active lifesty...
139258    engineering|science|technology|education|histo...
Name: tags, dtype: object

In [38]:
df.title.sample(4)

238125    My Friends Trapped me in PERMAFROST, so I got ...
43038     The ROG Phone 5 has a Problem - Durability Tes...
192082        J'ouvre un RESTAURANT 100% GRATUIT! | DENYZEE
237925                       Why Buildings Need Foundations
Name: title, dtype: object

In [39]:
df.channelTitle.sample(4)

174804                 I Talk
258894                  Ryscu
310189    Hailey Rhode Bieber
119310           yung maestro
Name: channelTitle, dtype: object

In [40]:
df.description.sample(4)

306425    The National Lottery draw results for Lotto re...
147227    This tiny house is as unique as it is stunning...
138695    Quick lil reaction to my before they were famo...
253819    God, thank you for your gift of life and your ...
Name: description, dtype: object

### The function defined below checks if the text contains non english characters or not

In [41]:
def isEnglish(s):
    try:
        s.encode('ascii')
    except UnicodeEncodeError:
        return False
    else:
        return True

In [42]:
isEnglish('slabiky, ale liší se podle významu')

False

In [43]:
isEnglish('This sentence is in English')

True

### Function to remove non english characters from the text:

In [44]:
def removeNonEnglishWords(text):
    filteredText = []
    for word in text.split():
        if isEnglish(word):
            filteredText.append(word)
    
    return " ".join(filteredText)
        

#### Validating the function 'removeNonEnglishWords'

In [45]:
text = "‣ what was tekoi:  tekoi commentary:   old version of tekoi:  crowdfundersbob kunz, john buchan, nevin spoljaric, donal botkin, bn-12, chris chapin, richard jenkins, phil gardner, martin, steven grimm, سليمان العقل, david f watson, colin millions, saki comandao, ben schwab, jason lewandowski, marco arment, shantanu raj, rictic, emptymachine, george lin, henry ng, thunda plum, awoo, david tyler, fuesu, iulus, jordan earls, joshua jamison, nick fish, nick gibson, tyler bryant, zach whittle, oliver steele, kermit norlund, kevin costello, derek bonner, derek jackson, mikko , orbit_junkie, ron bowes, tómas árni jónasson, bryan mclemore, alex simonides, felix weis, melvin sowah, christopher mutchler, giulio bontadini, paul alom, ryan tripicchio, scot melville, bear, chrysilis, david palomares, emil, erik parasiuk, esteban santana santana, freddi hørlyck, john rogers, leon, peter lomax, rhys parry, shiroiyami, tristan watts-willis, veronica peshterianu, dag viggo lokøen, john lee, maxime zielony, julien dubois, elizabeth keathley, nicholas welna## musicdavid rees:"
print(removeNonEnglishWords(text))

what was tekoi: tekoi commentary: old version of tekoi: crowdfundersbob kunz, john buchan, nevin spoljaric, donal botkin, bn-12, chris chapin, richard jenkins, phil gardner, martin, steven grimm, david f watson, colin millions, saki comandao, ben schwab, jason lewandowski, marco arment, shantanu raj, rictic, emptymachine, george lin, henry ng, thunda plum, awoo, david tyler, fuesu, iulus, jordan earls, joshua jamison, nick fish, nick gibson, tyler bryant, zach whittle, oliver steele, kermit norlund, kevin costello, derek bonner, derek jackson, mikko , orbit_junkie, ron bowes, bryan mclemore, alex simonides, felix weis, melvin sowah, christopher mutchler, giulio bontadini, paul alom, ryan tripicchio, scot melville, bear, chrysilis, david palomares, emil, erik parasiuk, esteban santana santana, freddi john rogers, leon, peter lomax, rhys parry, shiroiyami, tristan watts-willis, veronica peshterianu, dag viggo john lee, maxime zielony, julien dubois, elizabeth keathley, nicholas welna## m

#### Creating a flag to check if each record in description is in English

In [46]:
df['isEnglish'] = df.description.apply(lambda s: isEnglish(s))

In [47]:
df.isEnglish.value_counts()

False    257129
True     144798
Name: isEnglish, dtype: int64

#### False--- indicates the presence of non- English characters in description
#### True---- indicates all the words in its description are in English

In [48]:
df[df.isEnglish==False].description

0         SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
1         While running her own modding shop, Ramya Pare...
2         I left youtube for a month and this is what ha...
3         Subscribe to XXL → http://bit.ly/subscribe-xxl...
5         Subscribe To My Channel - https://www.youtube....
                                ...                        
401919    Undercover in a 100 player Fortnite hide and s...
401920    NCT DREAM's 2nd album repackage Beatbox is out...
401923    ►How I Stole Max Hearts On This Minecraft SMP....
401925    Welcome your faces back to a new dawn and a ne...
401926    Thanks to ExpressVPN for sponsoring this video...
Name: description, Length: 257129, dtype: object

#### adding a new feature des (with cleaned description ) into the dataframe

In [49]:
df['des']= df.description.apply(removeNonEnglishWords)

In [50]:
df[['description','des']].sample(5)

,description,des
60552,"1,000 DAYS HARDCORE VIDEO: https://youtu.be/Mh...","1,000 DAYS HARDCORE VIDEO: Here: a LIKE if you..."
165729,"Downfalls High. Premiering January 15, 2021 at...","Downfalls High. Premiering January 15, 2021 at..."
37413,"She may be a small town girl, but she’s got a ...","She may be a small town girl, but got a big vo..."
146283,A DEMON IN 6LUE Out Now: https://LilLoaded.lnk...,A DEMON IN 6LUE Out Now: https://LilLoaded.lnk...
36951,How to Complete the Hog Mountain Challenge in ...,How to Complete the Hog Mountain Challenge in ...


#### From the above code, des column has only the words in that are in English .So it is safe to drop Description and isEnglish(flag) column.

In [51]:
df.drop(columns=['description','isEnglish'],axis=1,inplace=True)

#### Now applying the same logic on tags, title and channel_title  columns to ensure the presence of just English words

##### Tags cleaning:

In [52]:
df['isEnglish'] = df.tags.apply(lambda s: isEnglish(s))
df[df.isEnglish==False].tags.count()# finding the number of tags that consists of non english characters

31726

##### adding a new feature c_tags (with cleaned tags ) into the dataframe

In [53]:
df['c_tags']= df.tags.apply(removeNonEnglishWords)

Now, the c_tags column has just the english words from tags column. hence, it is safe to drop isEnglish and tags columns from df

In [54]:
df.drop(columns=['isEnglish','tags'],axis=1,inplace=True)

##### Title(video title)

In [55]:
df['isEnglish'] = df.title.apply(lambda s: isEnglish(s))
df[df.isEnglish==False].title.count()

60210

In [56]:
df['c_title']= df.title.apply(removeNonEnglishWords)

#### The above code has cleaned tilte column and stored into c_title column and hence we are dropping isEnglish and tiltle column from df

In [57]:
df.drop(columns=['isEnglish','title'],axis=1,inplace=True)

##### Channel_Title

In [58]:
df['isEnglish'] = df.channelTitle.apply(lambda s: isEnglish(s))
df[df.isEnglish==False].channelTitle.count()

5483

In [59]:
df['channel_title']= df.channelTitle.apply(removeNonEnglishWords)

#### Dropping off isEnglish and channelTitle columns and keeping the cleaned channel_title

In [60]:
df.drop(columns=['channelTitle','isEnglish'],axis=1,inplace=True)

### Removing links and converting all the relevant text columns to lowercase

In [61]:
#remove links from the description
df.des= df.des.str.replace('http\S+|www.\S+',''\
                                     ,regex=True).str.replace('\r+',''\
                                     ,regex= True).str.lower()
#removing Punctuation from description
df.des = df.des.str.replace(r'[^\w\s]+', '')

In [62]:
#replacing | from tags with a space and converting the text to lowercase
df.c_tags= df.c_tags.str.replace('|', ' ').str.lower()
#removing punctuation if any from tags
df.c_tags = df.c_tags.str.replace(r'[^\w\s]+', '')
#Replacing None in tags to ''
df.c_tags = df.c_tags.str.replace('none','')
df[df.c_tags=='none'].head(1)

,video_id,publishedAt,categoryId,trending_date,view_count,likes,dislikes,comment_count,thumbnail_link,category_name,country,des,c_tags,c_title,channel_title


In [63]:
#replacing | from title with a space and converting the text to lowercase
df.c_title= df.c_title.str.replace('|', ' ').str.lower()
#remove links from the title
df.c_title= df.c_title.str.replace('http\S+|www.\S+',''\
                                     ,regex=True).str.replace('\r+',''\
                                     ,regex= True).str.lower()
#removing punctuation if any from title
df.c_title = df.c_title.str.replace(r'[^\w\s]+', '')

In [64]:
#replacing | from channel_title with a space 
df.channel_title= df.channel_title.str.replace('|', ' ')
#removing punctuation if any from title
df.channel_title = df.channel_title.str.replace(r'[^\w\s]+', '')

In [65]:
df.isna().any().sum()

0

### With this step all the text columns are cleaned to be in English and lower case without any links or puntuation or special characters in them.

In [66]:
df.sample(2)

,video_id,publishedAt,categoryId,trending_date,view_count,likes,dislikes,comment_count,thumbnail_link,category_name,country,des,c_tags,c_title,channel_title
370483,p9cCrwKhjGs,2021-12-23T23:00:10Z,10,2022-01-02T00:00:00Z,11198316,166448,0,5829,https://i.ytimg.com/vi/p9cCrwKhjGs/default.jpg,Music,Great Britain,stream or download here produced by lipa sbs ...,butrint imeri corazon butrint imeri korazon co...,butrint imeri ft don xhoni corazon,AVD Digital
343975,Z-5FQVspXeU,2021-08-19T16:37:39Z,20,2021-08-21T00:00:00Z,1185784,88465,311,7223,https://i.ytimg.com/vi/Z-5FQVspXeU/default.jpg,Gaming,Great Britain,please boop the like button if you enjoy the v...,ldshadowlady minecraft mini game girl gamer pi...,royal wedding ep 11 minecraft empires 117,LDShadowLady


### Converting Datatypes for Analysis


In [67]:
df.dtypes

video_id          object
publishedAt       object
categoryId         int64
trending_date     object
view_count         int64
likes              int64
dislikes           int64
comment_count      int64
thumbnail_link    object
category_name     object
country           object
des               object
c_tags            object
c_title           object
channel_title     object
dtype: object

#### 1. Converting published_at, trending_date to datetime and removing time zones
#### 2. Converting country to category

In [68]:
df['publishedAt']=pd.to_datetime(df.publishedAt)
df['publishedAt'] = df['publishedAt'].dt.tz_convert(None)
df['trending_date']=pd.to_datetime(df.trending_date)
df['trending_date'] = df['trending_date'].dt.tz_convert(None)
df['country']= df['country'].astype('category')

In [69]:
df.dtypes

video_id                  object
publishedAt       datetime64[ns]
categoryId                 int64
trending_date     datetime64[ns]
view_count                 int64
likes                      int64
dislikes                   int64
comment_count              int64
thumbnail_link            object
category_name             object
country                 category
des                       object
c_tags                    object
c_title                   object
channel_title             object
dtype: object

In [70]:
df.isna().any().sum()

0

### Rename the columns for easier understanding and uniformity

In [71]:
df.rename(columns={"publishedAt": "published_at","categoryId" : "category_id"\
                   ,'des':'description','c_tags':'tags','c_title':'video_title'}, inplace = True)

In [72]:
df.head(1)

,video_id,published_at,category_id,trending_date,view_count,likes,dislikes,comment_count,thumbnail_link,category_name,country,description,tags,video_title,channel_title
0,3C66w5Z0ixs,2020-08-11 19:20:14,22,2020-08-12,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,People & Blogs,USA,subscribe to brawadis follow me on social twi...,brawadis prank basketball skits ghost funny vi...,i asked her to be my girlfriend,Brawadis


### Data Correctness

###  We observed few records with likes and dislikes greater than  view_count, which is practically impossible.
##### Dropping records with likes > view_count.

In [73]:
df[df.likes>df.view_count].sample(2)

,video_id,published_at,category_id,trending_date,view_count,likes,dislikes,comment_count,thumbnail_link,category_name,country,description,tags,video_title,channel_title
15289,r7nYQXsxJdU,2020-10-25 01:40:31,24,2020-10-27,0,4257,384,240,https://i.ytimg.com/vi/r7nYQXsxJdU/default.jpg,Entertainment,USA,hbcu homecoming 2020 is going down we welcome ...,2 chainz desi banks lionel richie lance gross...,hbcu homecoming 2020 meet me on the yard,YouTube Originals
74267,Hb3rmh-_FMw,2021-08-10 15:04:25,27,2021-08-12,0,18653,1545,0,https://i.ytimg.com/vi/Hb3rmh-_FMw/default.jpg,Education,USA,epilepsy warning watch and create short videos...,,introducing the shorter side of youtube,YouTube


In [74]:
df.drop(df[df.likes>df.view_count].index,inplace=True)

Resetting index after dropping few rows

In [75]:
df.reset_index(drop=True, inplace=True)

### Writing this cleaned file to compressed csv

In [76]:
df.set_index('video_id',inplace=True)
df.to_csv('Clean_Dataset_final.csv.zip')

In [77]:
df.shape

(401912, 14)

### -------------------------------------Verified clean data for analysis---------------------------------------------------------


### Now the Dataset looks clean for some exploration

Resetting index back as we set index to Video_id for writing the file to csv

In [78]:
df.reset_index(inplace = True)

In [79]:
df.head(1)

,video_id,published_at,category_id,trending_date,view_count,likes,dislikes,comment_count,thumbnail_link,category_name,country,description,tags,video_title,channel_title
0,3C66w5Z0ixs,2020-08-11 19:20:14,22,2020-08-12,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,People & Blogs,USA,subscribe to brawadis follow me on social twi...,brawadis prank basketball skits ghost funny vi...,i asked her to be my girlfriend,Brawadis


## Cleaned Dataset Description

<ol>
    <li>video_id-------------------------	 Uniquely identifies each video</li>
    <li>published_at-----------------------    Date and Time of video published</li>
    <li>category_id------------------------    Id of category the video belongs to</li>
    <li>trending_date----------------------    Date and time when the video got to Trending</li>
    <li>view_count-------------------------    Number of views (cumulative)</li>
    <li>likes------------------------------	Number of Likes(cumulative)</li>
    <li>dislikes---------------------------    Number of dislikes(cumulative)</li>
    <li>comment_count----------------------   Number of comments(cumulative)</li>
    <li>category_name-----------------------	Name of category corresponding to Id</li>
    <li>country----------------------------------    Country in which the video was trending</li>
    <li>description-----------------------------   Description of video by the creator</li>
    <li>tags---------------------------------------   Tags of the video by the creator</li>
    <li>video_title------------------------------   Title of the video</li>
    <li>channel_title--------------------------	Channel Title of the video</li>
    <li>thumbnail_link--------------------------	link of the video thumbnail</li>
    
</ol>
